# Scrape Data from Shopee with beautifulsoup and selenium
### Data เพื่อการทำ Marketing ของผู้ขาย ซึ่งประกอบไปด้วย
- ชื่อสินค้าสินค้า ราคาสินค้าที่ขายจริง ยอดขายแต่ละสินค้า และจังหวัดที่เป็นต้นทางของสินค้า
- ไฟล์นี้เป็นการเขียนเองนะครับผม หากผิดพลาดประการใดแจ้งได้เลยครับ
# จัดทำโดย 
- ปวิตร ตุลวรรธนะ 6241133326

In [ ]:
!pip install selenium
!pip install bs4

# Code ที่ต้อง Import

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

Gecko Driver: https://github.com/mozilla/geckodriver/releases

# Selenium เพื่อการเข้า Web

In [ ]:
#Open browser ด้วย Selenium
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\COMPUTER\Desktop\Shopee_Web_Scraping\geckodriver.exe', options=options)
driver.get('https://shopee.co.th/')

In [ ]:
#เลือกภาษาไทย Selenium
thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
#time.sleep(3)

In [ ]:
#ปิดโฆษณา Selenium
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()

In [ ]:
#Search ใน Search bar ของ Shopee
search = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('computer')

In [ ]:
#กด Enter
search.send_keys(Keys.ENTER)

In [ ]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

# Scraping Process

In [ ]:
data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup

In [ ]:
soup.select(".row>div") #css selector

In [ ]:
el=soup.select(".row>div")

In [ ]:
len(el)

In [ ]:
e = el[0] #แทนกล่องของสินค้าแต่ละกล่อง

# Scrape ชื่อ

In [ ]:
#ดึงข้อมูลชื่อ
e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()

# Scrape ราคา

In [ ]:
#ราคา A baht - B baht, ราคาเต็ม
e.select_one("div.col-xs-2-4> a> div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()

In [ ]:
#ราคาลดแล้ว
p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
if p == '':
    print(1/0)
p

In [ ]:
#ราคารวมการตัด
try:
    p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
    if p == '':
        print(1/0)
except:
    p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
p

In [ ]:
#ตัด - และเลือกเลขด้านหน้า Range
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

In [ ]:
#รวม
try:
    p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
except:
    p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

# Scrape ยอดขาย

In [ ]:
sales = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
try:
    sales = float(re.sub('\D','',sales))
except:
    sales = 0
sales

# Scrape จังหวัด

In [ ]:
#จังหวัด
e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()

# Scrape ข้อมูลทั้งหมดที่ต้องการ

ทำเป็น list ของในหน้านั้น ๆ ที่กำลังดูอยู่ใน Web Shopee
นำ Code ต่าง ๆ มารวมกัน

In [ ]:
records=[]
data = driver.page_source
soup = bs4.BeautifulSoup(data)
el=soup.select(".row>div")
for e in el:
    name = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
    try:
        p = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
        if p == '':
            print(1/0)
    except:
        p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
    p = p.replace(",", "").replace("฿", "").replace(" ", "")
    try:
        start_price = float(p)
    except:
        price = p.rpartition('-')[0]
        start_price = float(price)

    sales = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
    try:
        sales = float(re.sub('\D','',sales))
    except:
        sales = 0
    province = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
    #print(name, p)
    records.append([name, start_price,sales,province])

records

In [ ]:
len(records)

In [ ]:
#ใช้ Selenium ในการกดปุ่มเพื่อไปหน้าถัดไป
next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[8]')
next_button.click()

In [ ]:
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
records=[]
for i in range(3):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select(".row>div")
    for e in el:
        name = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])


    time.sleep(5)
    next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[8]')
    next_button.click()
    time.sleep(5)


In [ ]:
#ทำเป็น DataFrame
df = pd.DataFrame(records, columns=['itemname','price','qty','province']) # ใส้ชื่อ Columns ทั้งหมด
df

# สูตรที่สามารถนำไปใช้ได้

In [ ]:
# ตั้งแต่เริ่มจนจบ
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe' #เปลี่ยนตาม Location ของไฟล์ Firefox ของท่านเอง
driver = webdriver.Firefox(executable_path=r'C:\Users\COMPUTER\Desktop\Shopee_Web_Scraping\geckodriver.exe', options=options) #เปลี่ยนตาม Location Gecko Driver ของท่านเอง
driver.get('https://shopee.co.th/')
time.sleep(5)
thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
time.sleep(3)
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()
time.sleep(3)
search = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('dog') #ใส่เนื้อหาที่ต้องการที่จะ Scrape จากเว็บ Shopee
search.send_keys(Keys.ENTER)

driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

time.sleep(5)
records=[]
for i in range(3): #ใส่จำนวนหน้าที่ต้องการ Scrape ในวงเล็บ
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select(".row>div")
    for e in el: #เพิ่มเนื้อหาต่าง ๆ ที่จะ Scrape ใน Loop นี้
        name = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])


    time.sleep(5)
    next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[8]')
    next_button.click()
    time.sleep(5)

df = pd.DataFrame(records, columns=['itemname','price','qty','province'])
df

In [ ]:
df.to_excel('Shopee_Computer.xlsx', encoding='utf-8')